# Cleaning DDoS CICFlowMeter Data

In [1]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pickle

### Loading IDS2017 DDoS data from Wednesday converted by CICFlowMeter

In [2]:
data = pd.read_csv(
    './data/IDS2017Data/netflow_data/Wednesday-workingHours.pcap_ISCX.csv')
data.shape

/home/talhajavaid32/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(692703, 85)

In [3]:
# Sample some
data.sample(20)

Flow ID       Source IP  \
185525       172.16.0.1-192.168.10.50-42504-80-6      172.16.0.1   
77343        172.16.0.1-192.168.10.50-53518-80-6      172.16.0.1   
418866     192.168.10.3-192.168.10.8-53-54209-17    192.168.10.8   
586468    192.168.10.3-192.168.10.12-53-59220-17   192.168.10.12   
8745      192.168.10.17-52.42.178.188-35754-80-6   192.168.10.17   
199069       172.16.0.1-192.168.10.50-60432-80-6      172.16.0.1   
282553       172.16.0.1-192.168.10.50-32882-80-6      172.16.0.1   
7358      192.168.10.19-91.189.91.157-123-123-17   192.168.10.19   
3886       192.168.10.5-52.71.99.142-49772-443-6    52.71.99.142   
369782     192.168.10.1-192.168.10.3-53-60455-17    192.168.10.3   
604094     192.168.10.3-192.168.10.5-53-55130-17    192.168.10.5   
190642       172.16.0.1-192.168.10.50-49548-80-6      172.16.0.1   
87252        172.16.0.1-192.168.10.50-33988-80-6      172.16.0.1   
117234       172.16.0.1-192.168.10.50-33396-80-6      172.16.0.1   
556426  182.253.240.30-192.168.10.51-443-36882-6  182.253.240.30   
72423        172.16.0.1-192.168.10.50-37964-80-6      172.16.0.1   
460709  151.101.20.207-192.168.10.14-443-56590-6   192.168.10.14   
214108       172.16.0.1-192.168.10.50-53888-80-6      172.16.0.1   
88073        172.16.0.1-192.168.10.50-34668-80-6      172.16.0.1   
255401       172.16.0.1-192.168.10.50-53386-80-6      172.16.0.1   

         Source Port  Destination IP   Destination Port   Protocol  \
185525         42504   192.168.10.50                 80          6   
77343          53518   192.168.10.50                 80          6   
418866         54209    192.168.10.3                 53         17   
586468         59220    192.168.10.3                 53         17   
8745           35754   52.42.178.188                 80          6   
199069         60432   192.168.10.50                 80          6   
282553         32882   192.168.10.50                 80          6   
7358             123   91.189.91.157                123         17   
3886             443    192.168.10.5              49772          6   
369782         60455    192.168.10.1                 53         17   
604094         55130    192.168.10.3                 53         17   
190642         49548   192.168.10.50                 80          6   
87252          33988   192.168.10.50                 80          6   
117234         33396   192.168.10.50                 80          6   
556426           443   192.168.10.51              36882          6   
72423          37964   192.168.10.50                 80          6   
460709         56590  151.101.20.207                443          6   
214108         53888   192.168.10.50                 80          6   
88073          34668   192.168.10.50                 80          6   
255401         53386   192.168.10.50                 80          6   

             Timestamp   Flow Duration   Total Fwd Packets  \
185525  5/7/2017 10:50        86105207                   9   
77343   5/7/2017 10:43             985                   2   
418866   5/7/2017 1:53           30657                   2   
586468   5/7/2017 4:06          133053                   2   
8745     5/7/2017 9:07        60591961                   9   
199069  5/7/2017 10:51        97785937                   6   
282553  5/7/2017 10:57       100727611                   8   
7358     5/7/2017 9:01        66036467                   2   
3886     5/7/2017 8:55           29456                   3   
369782  5/7/2017 12:56          226923                   1   
604094   5/7/2017 2:58           24011                   2   
190642  5/7/2017 10:50        99430851                   6   
87252   5/7/2017 10:43           24875                   3   
117234  5/7/2017 10:45        98166860                   7   
556426   5/7/2017 9:07              10                   1   
72423   5/7/2017 10:25       109239510                   4   
460709   5/7/2017 3:18         5109603                   9   
214108  5/7/2017 10:52   

#### Fix column headers

In [4]:
data.columns = data.columns.str.lower().str.strip().str.replace(
    ' ', '_').str.replace('/s', '').str.replace('/', '_').str.replace('.', '_')

In [5]:
cols = list(data.columns)
cols.remove('fwd_header_length_1')

In [6]:
with open('./experiments/columns.p', 'wb') as p:
    pickle.dump(cols, p)

In [7]:
# Column 20 and 21 have mixed type: convert to float64
data.iloc[:,[20,21]] = data.iloc[:,[20,21]].astype('float64')

### Filter out heartbleed

In [8]:
data = data[data.label != 'Heartbleed']

#### Features

In [9]:
# Get list of cols having type object
object_cols = list(data.dtypes[data.dtypes == 'object'].keys())
# Filter them out from all_cols list
all_cols = cols
feat = list(filter(lambda x: x not in object_cols, all_cols))

In [10]:
cols = ['flow_id'] + feat + ['label'] # Columns to keep from data
data = data[cols]

#### Imputing NaNs

In [11]:
# Rows containing either NaNs or Inf
(data.isnull() | (data == float('Inf'))).sum().max()

1297

In [12]:
# Dropping NaNs and Infs
data = data[~(data.isnull().any(axis=1) | (data == float('Inf')).any(axis=1))]

In [13]:
print 'No NaNs' if sum(data.isnull().any()) == 0 else 'Found NaNs'

No NaNs


In [14]:
print 'No Infs' if sum((data == float('Inf')).any()) == 0 else 'Found Infs'

No Infs


## Writing clean data

In [15]:
data.to_csv('./data/IDS2017Data/DDoS-WednesdayData/CICFlowData_clean.csv', index=False)

## Clean netflow data written to data/IDS2017Data/DDoS-WednesdayData/CICFlowData_clean.csv